# MARKET SCR step-by-step

EIOPA 2025: https://eur-lex.europa.eu/legal-content/EN/TXT/?uri=CELEX:02009L0138-20250117
- Information on Market Risk Submodule is in Annex IV. It links to Article 105(5)

In [131]:
import numpy as np
import pandas as pd
import math as m

In [132]:
'''
cc_step: rating
0 : "AAA"
1 : "AA"
2 : "A"
3 : "BBB"
4 : "BB"
5 : "B"
6 : "CCC or lower"
7 : "Unrated"

exposure_types:
1 : "EEA governments and central banks denominated and funded in the domestic currency, multilateral banks and international organisations"
2 : "Standard"
3 : "Mortgage covered bond or public sector covered bonds exposure"
4 : "Property exposure"
5 : "non-EEA governments and central banks denominated and funded in the domestic currency exposures"
'''

'\ncc_step: rating\n0 : "AAA"\n1 : "AA"\n2 : "A"\n3 : "BBB"\n4 : "BB"\n5 : "B"\n6 : "CCC or lower"\n7 : "Unrated"\n\nexposure_types:\n1 : "EEA governments and central banks denominated and funded in the domestic currency, multilateral banks and international organisations"\n2 : "Standard"\n3 : "Mortgage covered bond or public sector covered bonds exposure"\n4 : "Property exposure"\n5 : "non-EEA governments and central banks denominated and funded in the domestic currency exposures"\n'

## Gathering data

In [133]:
portfolio_data = {
    "Asset Class": [ "World Equity", "Private Equity", "Real Estate", "Government Bonds",
        "Investment Grade Corporate Bonds", "High Yield Corporate Bonds", "Money Market"],
    
    "Index Used": ["MSCI World", "LPX50", "FTSE EPRA Nareit Developed Europe", "FTSE EMU Government Bond",
        "ICE BofA Global Corporate (GOBC)", "ICE BofA Global HY (HW00)", "EONIA & €STR"],
    
    "Short Name": ["Equity", "PE", "RE", "GB", "IG CB", "HY CB", "Money"],
    
    "Exposure Type": ["type1", "type2", "single_property", "gov_eea", "bonds", "bonds", "gov_eea"],
    
    "Credit Quality Step": [None, None, None, 1, 2, 5, None],
    
    "Duration": [None, None, None, 7.29, 5.90, 3.14, None]  
}